In [2]:
# bow vs tfidf

# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os

import dagshub

mlflow.set_tracking_uri('https://dagshub.com/2003HARSH/Text-Classification-using-MLOps.mlflow')
dagshub.init(repo_owner='2003HARSH', repo_name='Text-Classification-using-MLOps', mlflow=True)

# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

# Normalize the text data
df = normalize_text(df)

df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

# Set the experiment name
mlflow.set_experiment("Bow vs TfIdf")

# Define feature extraction methods
vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

# Define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

# Start the parent run
with mlflow.start_run(run_name="All Experiments") as parent_run:
    # Loop through algorithms and feature extraction methods (Child Runs)
    for algo_name, algorithm in algorithms.items():
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                X = vectorizer.fit_transform(df['content'])
                y = df['sentiment']
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                

                # Log preprocessing parameters
                mlflow.log_param("vectorizer", vec_name)
                mlflow.log_param("algorithm", algo_name)
                mlflow.log_param("test_size", 0.2)
                
                # Model training
                model = algorithm
                model.fit(X_train, y_train)
                
                # Log model parameters
                if algo_name == 'LogisticRegression':
                    mlflow.log_param("C", model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param("alpha", model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("max_depth", model.max_depth)
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                    mlflow.log_param("max_depth", model.max_depth)
                
                # Model evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                
                # Log evaluation metrics
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)
                
                #signature
                signature=mlflow.models.infer_signature(X_train,model.predict(X_train))

                # Log model
                mlflow.sklearn.log_model(model, "model",signature=signature)

                #Track the dataset
                dataset=mlflow.data.from_pandas(df)
                mlflow.log_input(dataset,"dataset")
                mlflow.log_param("random_state","42")
                
                # Save and log the notebook
                import os
                notebook_path = "experiment_2_BOW_vs_TFIDF.ipynb"
                os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
                mlflow.log_artifact(notebook_path)      

                # Print the results for verification
                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")

Initialized MLflow to track repo "2003HARSH/Text-Classification-using-MLOps"

Repository 2003HARSH/Text-Classification-using-MLOps initialized!

C:\Users\harsh\AppData\Local\Temp\ipykernel_11336\2927107986.py:87: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
c:\Users\harsh\Desktop\Text-Classification-using-MLOps\venv\lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) wheneve

Algorithm: LogisticRegression, Feature Engineering: BoW
Accuracy: 0.7951807228915663
Precision: 0.7858527131782945
Recall: 0.7990147783251231
F1 Score: 0.7923790913531998


2024/11/04 19:14:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run LogisticRegression with BoW at: https://dagshub.com/2003HARSH/Text-Classification-using-MLOps.mlflow/#/experiments/2/runs/877726825e0e43b88bac472f8e5ee415.
2024/11/04 19:14:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/2003HARSH/Text-Classification-using-MLOps.mlflow/#/experiments/2.
c:\Users\harsh\Desktop\Text-Classification-using-MLOps\venv\lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these c

...

2024/11/04 19:20:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest with TF-IDF at: https://dagshub.com/2003HARSH/Text-Classification-using-MLOps.mlflow/#/experiments/2/runs/05e4c13aa79b4394af44734c4ccb6d09.
2024/11/04 19:20:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/2003HARSH/Text-Classification-using-MLOps.mlflow/#/experiments/2.
c:\Users\harsh\Desktop\Text-Classification-using-MLOps\venv\lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these colu

Algorithm: GradientBoosting, Feature Engineering: BoW
Accuracy: 0.7272289156626506
Precision: 0.7997329773030708
Recall: 0.5901477832512315
F1 Score: 0.6791383219954649


2024/11/04 19:21:10 INFO mlflow.tracking._tracking_service.client: 🏃 View run GradientBoosting with BoW at: https://dagshub.com/2003HARSH/Text-Classification-using-MLOps.mlflow/#/experiments/2/runs/22b4b7fba2aa4913a80d745a1048005d.
2024/11/04 19:21:10 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/2003HARSH/Text-Classification-using-MLOps.mlflow/#/experiments/2.
c:\Users\harsh\Desktop\Text-Classification-using-MLOps\venv\lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these col

Algorithm: GradientBoosting, Feature Engineering: TF-IDF
Accuracy: 0.7257831325301205
Precision: 0.8071625344352618
Recall: 0.5773399014778325
F1 Score: 0.6731763354394027


2024/11/04 19:22:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run GradientBoosting with TF-IDF at: https://dagshub.com/2003HARSH/Text-Classification-using-MLOps.mlflow/#/experiments/2/runs/9ad0a0ce22c54689af49bc7522047f23.
2024/11/04 19:22:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/2003HARSH/Text-Classification-using-MLOps.mlflow/#/experiments/2.
2024/11/04 19:22:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run All Experiments at: https://dagshub.com/2003HARSH/Text-Classification-using-MLOps.mlflow/#/experiments/2/runs/352bcfe8aa7f48638c9a98e0719a9b70.
2024/11/04 19:22:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/2003HARSH/Text-Classification-using-MLOps.mlflow/#/experiments/2.
